# Actividad 2.2 Preprocesado de los datos
**Daniel Cruz Arciniega A01701370**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Preparar los datos

In [6]:
labels_csv = pd.read_csv("/content/drive/MyDrive/Análisis de datos/Dog Vision/labels.csv")
labels_csv.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [7]:
labels_csv.describe()

,id,breed
count,10222,10222
unique,10222,120
top,000bec180eb18c7604dcecc8fe0dba07,scottish_deerhound
freq,1,126


Guardar las rutas de las imágenes en el arreglo `filenames`

In [8]:
filenames = ["/content/drive/MyDrive/Análisis de datos/Dog Vision/train/" + fname + ".jpg" for fname in labels_csv["id"]]

filenames[:10]

['/content/drive/MyDrive/Análisis de datos/Dog Vision/train/000bec180eb18c7604dcecc8fe0dba07.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/001cdf01b096e06d78e9e5112d419397.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/00214f311d5d2247d5dfe4fe24b2303d.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/0021f9ceb3235effd7fcde7f7538ed62.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/002211c81b498ef88e1b40b9abf84e1d.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/00290d3e1fdd27226ba27a8ce248ce85.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/002a283a315af96eaea0e28e7163b21b.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/003df8b8a8b05244b1d920bb6cf451f9.jpg',
 '/content/drive/MyDrive/Análisis de datos/Dog Vision/train/0042188c895a2f14ef64a918ed9c7b64.jpg']

In [9]:
labels = labels_csv["breed"].to_numpy()
labels

array(['boston_bull', 'dingo', 'pekinese', ..., 'airedale',
       'miniature_pinscher', 'chesapeake_bay_retriever'], dtype=object)

In [10]:
len(labels) == len(filenames)

True

In [11]:
# Guardar los valores únicos de razas
unique_breeds = np.unique(labels)
len(unique_breeds)

120

In [12]:
# Arreglo de booleanos
print(labels[0])
labels[0] == unique_breeds

boston_bull


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

Convertir cada raza en un arreglo de booleanos para simplificar las clases en unos y ceros

In [13]:
bool_labels = [label == unique_breeds for label in labels]
len(bool_labels)

10222

In [14]:
print(labels[0]) # raza
print(np.where(unique_breeds == labels[0])) # indice en donde se encuentra la raza
print(bool_labels[0].argmax()) # indice donde se encuentra la raza en el arreglo booleano
print(bool_labels[0].astype(int)) # arreglo booleano con 0 y 1

boston_bull
(array([19]),)
19
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


## Crear datos de entrenamiento y de prueba
Se usarán X_val y y_val para validar todos los datos de entrenamiento porque los datos de prueba se encuentran en la carpeta `test` y no vienen con etiquetas.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X = filenames
y = bool_labels

X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(8177, 8177, 2045, 2045)

## Preprocesamiento de datos
En esta sección se convertirán las imágenes a formato numérico y se crearán batches a partir de los datos.

In [17]:
import tensorflow as tf

### Preprocesamiento de imágenes
Convertir las imágenes en formato numérico usando la librería **Tensorflow**.

Convertir la imagen a tamaño 224x224.

In [18]:
# Tamaño de imagen
IMG_SIZE = 224

# Función para preprocesamiento de imágenes
def process_image(image_path):
  # Leer el archivo de imagen
  image = tf.io.read_file(image_path)
  # Convertir la imagen a formato numérico con RGB
  image = tf.image.decode_jpeg(image, channels=3)
  # Convertir los valores RGB de 0-255 a 0-1
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Cambiar el tamaño de imagen a (224, 224)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])

  return image

In [19]:
# Probar la función con la primera imagen
test_img = process_image(filenames[0])
test_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.5705322 , 0.33588538, 0.15008473],
        [0.66901225, 0.46011657, 0.26022965],
        [0.61062384, 0.43763596, 0.21601713],
        ...,
        [0.8182312 , 0.7297958 , 0.26963907],
        [0.72454786, 0.6059109 , 0.2653059 ],
        [0.7494603 , 0.6252918 , 0.32456532]],

       [[0.6761415 , 0.4733761 , 0.3064293 ],
        [0.5672121 , 0.38192675, 0.20349538],
        [0.5883547 , 0.4322571 , 0.2403155 ],
        ...,
        [0.9326942 , 0.8260503 , 0.41093278],
        [0.86072475, 0.74250627, 0.3694122 ],
        [0.76781666, 0.6461604 , 0.29291284]],

       [[0.46350825, 0.29595745, 0.1536156 ],
        [0.42554107, 0.27039403, 0.12009467],
        [0.7887875 , 0.6547362 , 0.49015   ],
        ...,
        [0.89372385, 0.76964   , 0.41811958],
        [0.88609535, 0.7585918 , 0.36495727],
        [0.8243601 , 0.69494826, 0.2930575 ]],

       ...,

       [[0.7344025 , 0.4716574 , 0.2559711 ],
        [0.71

In [20]:
test_img.shape

TensorShape([224, 224, 3])

### Convertir los datos en batches
Convertir los datos en batches de tamaño 32 para acelerar el proceso de ajuste del modelo.

También se necesitarán los datos en formato de tupla (imagen, etiqueta) para que tensorflow funcione correctamente.

In [22]:
BATCH_SIZE = 32

# Regresa una tupla (image, label)
def get_image_label(image_path, label):
  image = process_image(image_path)
  return image, label

# Convertir los datos en batches
def create_data_batches(X, y=None, valid_data=False, test_data=False):
  # No se necesitan etiquetas para datos de prueba
  if test_data:
    print("Datos de prueba")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  elif valid_data:
    print("Datos de validación")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),  # imagenes
                                               tf.constant(y))) # etiquetas
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    print("Datos de entrenamiento")
    # Convertir imagenes y etiquetas en numeros
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    #  Revolver las imagenes y etiquetas
    data = data.shuffle(buffer_size=len(X))

    # Crear tuplas (image, label)
    data = data.map(get_image_label)

    # Convertir a batches
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [23]:
# Crear batches de entrenamiento y prueba
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

Datos de entrenamiento
Datos de validación


In [25]:
train_data

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.bool, name=None))>

In [26]:
val_data

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.bool, name=None))>